## FLAIR BERT returned error because it needs maximum 512 token in the sequence, and we have many sequences more than this number  

In [4]:
import pandas as pd
dataset = pd.read_csv('../data/dataset.csv', sep='\t', header=None).applymap(str)
dataset.columns = ["Language","label","text"]
languagesData=[]
loc = 0
languages = dataset[dataset.columns[0]].unique()
for i in languages:
    name = languages[loc]+"Data" 
    globals()[name] = pd.DataFrame( dataset[dataset.Language == i])
    loc += 1

In [5]:
len(englishData)

4519

In [35]:
englishData.text[1]

'trump asks sec to mull half-year corporate filings vs. quarterlyu.s. president donald trump said on friday he had asked the u.s. securities and exchange commission to study the impact of allowing companies to file reports with the financial regulator every six months instead of every quarter.source:: '

In [33]:
def lower_words(text):
    text = text.str.lower()
    return text

englishData['text'] = lower_words(englishData['text'])
input_str = englishData['text']
print(englishData.head())


  Language        label                                               text
0  english  advertising  exclusive: u.s. government seeks facebook help...
1  english  advertising  trump asks sec to mull half-year corporate fil...
2  english  advertising  wall st. up on trade hopes, s&p equals longest...
3  english  advertising  asian shares hit one-year low on turkey, china...
4  english  advertising  asian stocks weaken as turkey worries weigh, d...


In [34]:

import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
def remove_surl(text):
    text = text.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
    return text

def remove_url(text):
    text = text.apply(lambda x: re.split('http:\/\/.*', str(x))[0])
    return text

englishData['text'] = remove_url(englishData['text'])
englishData['text'] = remove_surl(englishData['text'])

englishData.head()

,Language,label,text
0,english,advertising,exclusive: u.s. government seeks facebook help...
1,english,advertising,trump asks sec to mull half-year corporate fil...
2,english,advertising,"wall st. up on trade hopes, s&p equals longest..."
3,english,advertising,"asian shares hit one-year low on turkey, china..."
4,english,advertising,"asian stocks weaken as turkey worries weigh, d..."


In [36]:
def remove_numbers(text):
    text = text.str.replace('\d+', '')
    return text


englishData['text'] = remove_numbers(englishData['text'])
englishData.head()

,Language,label,text
0,english,advertising,exclusive: u.s. government seeks facebook help...
1,english,advertising,trump asks sec to mull half-year corporate fil...
2,english,advertising,"wall st. up on trade hopes, s&p equals longest..."
3,english,advertising,"asian shares hit one-year low on turkey, china..."
4,english,advertising,"asian stocks weaken as turkey worries weigh, d..."


In [37]:
def remove_punctuations(text):
    text = text.str.replace('[^\w\s]','')
    return text


englishData['text'] = remove_punctuations(englishData['text'])
englishData.head()

,Language,label,text
0,english,advertising,exclusive us government seeks facebook help to...
1,english,advertising,trump asks sec to mull halfyear corporate fili...
2,english,advertising,wall st up on trade hopes sp equals longest bu...
3,english,advertising,asian shares hit oneyear low on turkey china w...
4,english,advertising,asian stocks weaken as turkey worries weigh do...


In [38]:

def remove_blank_space(col):
    col = col.str.strip()
    col = col.replace('\s+', ' ', regex=True)   
    return col


englishData['text'] = remove_blank_space(englishData.text)
englishData.head()

,Language,label,text
0,english,advertising,exclusive us government seeks facebook help to...
1,english,advertising,trump asks sec to mull halfyear corporate fili...
2,english,advertising,wall st up on trade hopes sp equals longest bu...
3,english,advertising,asian shares hit oneyear low on turkey china w...
4,english,advertising,asian stocks weaken as turkey worries weigh do...


In [39]:
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
def remove_stop_words(text, stop):    
    text.apply(lambda x: [item for item in x if item not in stop])
    return text

stop = stopwords.words('english')
remove_stop_words(englishData['text'],stop)
englishData.head()

,Language,label,text
0,english,advertising,exclusive us government seeks facebook help to...
1,english,advertising,trump asks sec to mull halfyear corporate fili...
2,english,advertising,wall st up on trade hopes sp equals longest bu...
3,english,advertising,asian shares hit oneyear low on turkey china w...
4,english,advertising,asian stocks weaken as turkey worries weigh do...


In [40]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


porter_stemmer = PorterStemmer()
englishData['text'] = englishData['text'].apply(stem_sentences)
englishData.head()

,Language,label,text
0,english,advertising,exclus us govern seek facebook help to wiretap...
1,english,advertising,trump ask sec to mull halfyear corpor file vs ...
2,english,advertising,wall st up on trade hope sp equal longest bull...
3,english,advertising,asian share hit oneyear low on turkey china wo...
4,english,advertising,asian stock weaken as turkey worri weigh dolla...


In [41]:
englishData = englishData[["label","text"]]
englishData['label'] = '__label__' + englishData['label'].astype(str)


In [6]:

englishData.iloc[0:int(len(englishData)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.8):int(len(englishData)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [29]:
englishData.head()

,label,text
0,__label__advertising,None
1,__label__advertising,None
2,__label__advertising,None
3,__label__advertising,None
4,__label__advertising,None


In [13]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings,DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
2019-05-11 19:47:05,957 Reading data from .
2019-05-11 19:47:05,958 Train: train.csv
2019-05-11 19:47:05,959 Dev: dev.csv
2019-05-11 19:47:05,960 Test: test.csv


In [14]:
len(corpus.dev)

452

In [15]:
# word_embeddings = [WordEmbeddings('glove'),FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
from flair.embeddings import BertEmbeddings

word_embeddings = [BertEmbeddings('bert-large-cased')]



In [16]:
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)


In [17]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)


In [18]:
trainer = ModelTrainer(classifier, corpus)


In [19]:
trainer.train('./', max_epochs=10, patience=1)

2019-05-11 19:47:50,038 ----------------------------------------------------------------------------------------------------
2019-05-11 19:47:50,039 Evaluation method: MICRO_F1_SCORE
2019-05-11 19:47:50,044 ----------------------------------------------------------------------------------------------------


ValueError: Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (2948 > 512). Running this sequence through BERT will result in indexing errors